# 1.) Import the Credit Card Fraud Data From CCLE

In [1]:
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

In [2]:
drive.mount('/content/gdrive/', force_remount = True)

Mounted at /content/gdrive/


In [3]:
df = pd.read_csv("/content/gdrive/MyDrive/441B/fraudTest.csv")

In [4]:
df.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0


In [5]:
df.columns

Index(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category',
       'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time',
       'merch_lat', 'merch_long', 'is_fraud'],
      dtype='object')

# 2.) Select four columns to use as features (one just be trans_date_trans)

In [6]:
df_select = df[['trans_date_trans_time', 'category', 'amt', 'city_pop', 'is_fraud']]

In [7]:
df_select.head()

,trans_date_trans_time,category,amt,city_pop,is_fraud
0,2020-06-21 12:14:25,personal_care,2.86,333497,0
1,2020-06-21 12:14:33,personal_care,29.84,302,0
2,2020-06-21 12:14:53,health_fitness,41.28,34496,0
3,2020-06-21 12:15:15,misc_pos,60.05,54767,0
4,2020-06-21 12:15:17,travel,3.19,1126,0


# 3.) Create a unique variable out of trans_date.

In [8]:
type(df_select['trans_date_trans_time'][0])

str

In [9]:
df_select['trans_date_trans_time'] = pd.to_datetime(df_select['trans_date_trans_time'])

<ipython-input-9-035beec6fe23>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['trans_date_trans_time'] = pd.to_datetime(df_select['trans_date_trans_time'])


In [10]:
df_select['hour']=[i.hour for i in df_select['trans_date_trans_time']]
df_select['hour'].head()

<ipython-input-10-b9add3657b11>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['hour']=[i.hour for i in df_select['trans_date_trans_time']]


0    12
1    12
2    12
3    12
4    12
Name: hour, dtype: int64

In [11]:
# 24 hours are classfied to 3 time intervals, morning, day and evening.
df_select['evening'] = np.where(df_select['hour']>=18, 1,0)
df_select['morning'] = np.where(df_select['hour']<=5, 1,0)
df_select.drop('trans_date_trans_time', axis =1)
dummies = pd.get_dummies(df['category'])
mergedf = pd.concat([df_select,dummies], axis=1)
df_select=mergedf.drop('category',axis=1)

<ipython-input-11-759f65311ce4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['evening'] = np.where(df_select['hour']>=18, 1,0)
<ipython-input-11-759f65311ce4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['morning'] = np.where(df_select['hour']<=5, 1,0)


In [12]:
df_select =df_select.drop(["trans_date_trans_time"], axis = 1)

In [13]:
df_select

,amt,city_pop,is_fraud,hour,evening,morning,entertainment,food_dining,gas_transport,grocery_net,grocery_pos,health_fitness,home,kids_pets,misc_net,misc_pos,personal_care,shopping_net,shopping_pos,travel
0,2.86,333497,0,12,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,29.84,302,0,12,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,41.28,34496,0,12,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,60.05,54767,0,12,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,3.19,1126,0,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555714,43.77,519,0,23,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
555715,111.84,28739,0,23,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
555716,86.88,3684,0,23,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
555717,7.99,129,0,23,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


# 4.) Oversample the data (this will be your training data).

# 5.) Train a Logistic regression.

In [14]:
train_size= int(len(df_select)*0.75)

target=df_select["is_fraud"]


In [15]:
feature = df_select.drop(["is_fraud"], axis = 1)

In [16]:
intarget = target[:train_size]
outtarget=target[train_size:]
indata=feature[:train_size]
outdata=feature[train_size:]

In [17]:
model = LogisticRegression().fit(indata, intarget)

# 6.) The company you are working for wants to target at a False Positive rate of 5% what threshold should you use? (Use oversampled data)

In [18]:
from sklearn.metrics import confusion_matrix

# Train your model
clf = LogisticRegression()
clf.fit(indata, intarget)

# Predict probabilities for test set
probs = clf.predict_proba(outdata)

# Find the threshold that gives a False Positive rate of 5%
threshold = 0
for t in np.arange(0, 1, 0.01):
    preds = probs[:,1] > t
    cm = confusion_matrix(outtarget, preds)
    fpr = cm[0,1] / (cm[0,0] + cm[0,1])
    if fpr <= 0.05:
        threshold = t
        break

In [19]:
print(threshold)

0.02


# 7.) If the company makes .02*amt on True transactions and loses -amt on False (Use original data)

In [20]:
outdata["Preds"] = preds
outdata["TP"] = outdata["Preds"] * outtarget

<ipython-input-20-773ba6a48fbe>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outdata["Preds"] = preds
<ipython-input-20-773ba6a48fbe>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outdata["TP"] = outdata["Preds"] * outtarget


In [21]:
outdata["TP"]

416789    0
416790    0
416791    0
416792    0
416793    0
         ..
555714    0
555715    0
555716    0
555717    0
555718    0
Name: TP, Length: 138930, dtype: int64

In [23]:
df_compare = pd.concat([outdata, outtarget], axis = 1)
df_compare['Preds'] = [0 if i == False else 1 for i in df_compare['Preds']]

In [24]:
# Create a column for false negative
df_compare['FN']= [0 if i==False else 1 for i in (df_compare['is_fraud']==1)&(df_compare['Preds']==0)]
# Add TP to df_compare
df_compare['TP']= outdata["TP"]

In [25]:
df_compare['Profit(+)/Loss(-)']= 0.02*df_compare['amt']*df_compare["TP"] + df_compare['FN']*-df_compare['amt']
df_compare['Profit(+)/Loss(-)'].sum()

-125556.01019999999

The company will suffer a loss of $125,556 if they use this logistic regression to predict fraud transactions.

# 8.) Using Logistic Regression Lasso to inform you. Would you use the selected features in a trusted prediction model?

In [26]:
model = LogisticRegression(solver='liblinear', penalty='l1').fit(indata, intarget)
model.coef_

array([[ 2.35473895e-03, -7.00112860e-07,  1.71432948e-01,
         1.80896022e+00,  3.19945458e+00, -4.96242876e+00,
        -5.04579881e+00, -2.89538722e+00, -2.95471032e+00,
        -1.82671558e+00, -5.10771538e+00, -5.37927970e+00,
        -5.15420779e+00, -2.42166195e+00, -4.19053624e+00,
        -4.85606854e+00, -3.03578232e+00, -4.25481286e+00,
        -8.70923128e+00]])

As you can see, the coefficients are not close to 0, so the selected features could be used in a trusted prediction model.
